## Binary IBDM
This notebook shows how to do image based data mining against a binary outcome. The outcome could be whatever you like, provided it is binary; some examples include overall survival, locoregional failure and feeding tube insertion.

First, we will install and load a few packages we will need

In [ ]:
%pip install pandas

We will also download the HNSCC data and unzip it

# NOTE: you might not need to do this is you end up on the same runtime as in the registration example, check the file browser on the left

In [ ]:
!wget https://www.dropbox.com/s/msb4uiq8rw79rqe/HNSCC_data.zip?dl=0 -O ./HNSCC_data.zip
!unzip HNSCC_data.zip
!rm HNSCC_data.zip

In [ ]:
## Import libraries we want and set up the notebook
import os
import time
import os.path
import numpy as np
import pandas as pd
try:
    from tqdm import tqdm_notebook as tqdm
    haveTQDM = True
except:
    haveTQDM = False

print(haveTQDM)

import SimpleITK as sitk
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

%matplotlib inline

## Make the notebook use full width of display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Getting started

To do an IBDM analysis, we need a few things:

- Registered data, so that the anatomy is (as far as possible) in the same spatial location
- Some kind of outcome to mine against
- A good understanding of any confounding variables in the dataset

We will use pandas to load a csv file of the data. Pandas is very powerful - here we will be using only its most basic functions.

In [ ]:
## Load the clinical data CSV
clinicalDataPath = "/content/HNSCC_data/clinicalData.csv"

clinicalData = pd.read_csv(clinicalDataPath)



Head and neck cancer is one of the more complex treatments, and often has several stages including surgery, chemotherapy alone, and concurrent chemo-RT. If we were to leave this complexity unaddressed we would probably introduce some bias, or leave some confounding information unaccounted for. Before doing anything, we will attempt to standardise a sub-cohort to do our IBDM in.

In the clinical data is a column recording what type of treatment a patient had. We can look at that column by indexing the dataframe with the column heading like this:

In [ ]:
clinicalData["Oncologic Treatment Summary"]

There are a lot of acronyms going on in this data, the most important are:

CMT  = Chemotherapy
ERT  = External beam radiotherapy
CCRT = Concurrent Chemo-radiotherapy

Other information includes what type of treatment was used when in a patient's pathway, for eaxmple:

`Surgery --> CMT --> CCRT + Cetuximab`

means a patient had surgery, then chemotherapy, then concurrent chemo-radiotherapy with an immunotherapy drug called Cetuximab.

The type of treatment a patient recieved is very important, because it affects how they respond. To remove potential confounding effects in our image based data mining, we would like to have all the patients be on as similar a course of treatment as possible. 

As a first set of patients to try, lets only take patients who started their treatment with CCRT. We will allow patients who had surgery after radiotherapy, because that shouldn't affect their response to the radiation. If we were being really strict, we could also exclude these patients/place some limit on when events occurred.

We can reduce our dataset using some pandas indexing tricks

In [ ]:
ccrtOnlyPatients = clinicalData[(clinicalData["Oncologic Treatment Summary"].str.contains('^CCRT', regex=True)) & (clinicalData["Oncologic Treatment Summary"].str.contains('\+', regex=True) == False)]
len(ccrtOnlyPatients["Oncologic Treatment Summary"])

To explain what is going on here:

- `[]` indexes the dataframe, getting values out of it. We can do this with a boolean mask 
- `(clinicalData["Oncologic Treatment Summary"].str.contains('^CCRT', regex=True))` asks "Does the value in this cell start with CCRT?" It says True where this is the case
- `(clinicalData["Oncologic Treatment Summary"].str.contains('\+', regex=True) == False)` Asks "Does the value in this cell not contain a + symbol?
- The `&` symbol combines the two logical statements into a boolean mask
- We end up with patients who have started their treatment with CCRT, and who didn't get immunotherapy

This reduces our original dataset quite a lot, now we only have 60 patients. After we've done some work here, maybe it would be a good idea to relax some of these requirements. You can choose how to relax them by changing the logical statements in this indexing.

So now we have our cohort, selected based on their radiotherapy. We need to check a few other things are in order before we continue:

- Do all the patients get the same overall dose?
- Do all the patients get the same number of fractions?
- Do we have all the outcome data we want to be able to do our datamining on these patients?

The total dose recieved by the patient is in our database in the column "RT Total Dose (Gy)". We can go ahead and histogram this for the patients we have selected.

In [ ]:
doses = ccrtOnlyPatients.hist(column="RT Total Dose (Gy)", bins=20) ## you can adjust many bits of this plot!
plt.xlabel("RT dose (Gy)");
plt.ylabel("# patients");

As you can see, most patients got a dose of 70 Gy, but there were a few who got higher or lower doses.

Let's look at the dose per fraction next by indexing it to get a table

In [ ]:
ccrtOnlyPatients[["Dose/Fraction (Gy/fx)", "RT Total Dose (Gy)", "Number of Fractions"]]


From this, we can see that all of the patients who got 72 Gy had a non-standard fractionation where they recieved some boost in the last couple of weeks of treatment. These patients always had their treatment in 40 fractions.

For simplicity, we will exclude these patients. 

In [ ]:
selectedPatients = ccrtOnlyPatients[ccrtOnlyPatients["Number of Fractions"].astype(int) < 40]
len(selectedPatients["Number of Fractions"])

Now we have a nice clean dataset where all our patients got nice easy to manipulate fractionation, we need to adjust their doses to take account of the different doses per fraction.

We have to do this because of how radiation works: 70 Gy in 33 fractions is a different biologically effective dose than 70 Gy in 35 fractions. If we don't adjust for this, then our results will probably not make any sense.

Different fractionations have diferent effects because of the different amount of time normal tissue has to recover. Before we can mine data with different fractionations, we need to standardize somehow. This is usually done with a biologically equivalent dose (BED) calculation, or an equivalent dose @ 2 Gy/fraction (EQD2) calculation. Here we will do a simple BED calculation, whereby we just multiply the real dose by a factor to take account of the differences in fractionation.

Also, whether we want to look at 'acute effects' (i.e. things that happen soon) or 'late effects' (i.e. things that happen after a few weeks/months) has a bearing on what we need to do here. The way these differences come into the BED calculation is in an $\alpha/\beta$ ratio. A ratio of $\alpha/\beta$ = 3.0 is usual when considering late effects.

Now we have to choose an endpoint. Let's look at feeding tube insertion, which I think is probably an early effect. That means we need an $\alpha/\beta$ of 10.

In [ ]:
## This function should create a voxelwise BED distribution. It makes the assumption that the dose in the plan is delivered in equal fractions
## This is obviously not 100% true due to setup uncertainty, motion, etc, but it is an ok assumtion for now

## BED = D_t * (1 + D_f/ab)  where:
## D_t is the total dose (per voxel)
## D_f = D_t/nFrac is the fraction dose per voxel
## ab is the alpha/beta ratio

def calculateVoxelwiseBED(dose, nFrac, alpha_beta=10.0):
    factor = 1.0 / (nFrac*alpha_beta)

    BED = dose*(1.0 + dose*factor)

    return BED

Now we have our BED correction, we can start loading data ready to do mining. We will load each image using SimpleITK, convert it to a numpy array and put it into a big numpy array. We will concurrently load the binary status from the clinical data as well and put that into a numpy array.

In the next cell, we will load all our data. Note that we pre-allocate the numpy array we will use to hold the data - this is a performance optimisation to make loading the data a bit quicker.

In [ ]:
dosesPath = "/content/HNSCC_data/warpedDoses/"
availableDoses = ["HNSCC-01-{0}".format(a.split('.')[0]) for a in os.listdir(dosesPath)]

availablePatientsMask = selectedPatients['ID'].isin(availableDoses)
probeDose = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(dosesPath, "{0:04d}.nii".format(int(2)))))

availablePatients = selectedPatients.loc[availablePatientsMask]
len(availablePatients)



doseArray = np.zeros((len(availablePatients), *probeDose.shape))
statusArray = np.zeros((len(availablePatients),))

print(doseArray.shape)


n = 0
for idx, pt in availablePatients.iterrows():
    dose_arr = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(dosesPath, f"{pt.ID.split('-')[-1]}.nii" ) ) )
    doseArray[n,...] = calculateVoxelwiseBED(dose_arr, pt["Number of Fractions"], alpha_beta=10.0)
    statusArray[n] = int(pt["Alive or Dead"] == "Dead")
    n += 1
    

Now we should have the dose and status data loaded, so we can start doing data mining!

Here we will defien a function to do binary data mining using a Student t-Test approach. This involves calculating the mean and variance in every voxel for each status group, then using that on a per-voxel basis to caluclate the per-voxel value of t. 

In [ ]:
def calculateT(doses, labels, mask=None):
    """
    Use scipy t test calculation to get a t map for this set of doses and labels

    Note, strictly we are doing welchs t test, which does not assume equal variance. Given this is per voxel, not assuming equal variance is the right thing to do
    """
    if mask is None:
        t_map, p_map = ttest_ind(doses[labels == 0], doses[labels == 1], axis=0, equal_var=False, nan_policy='omit')
    else:
        t_map, p_map = ttest_ind((doses*mask)[labels == 0], (doses*mask)[labels == 1], axis=0, equal_var=False, nan_policy='omit')

    return t_map, p_map

As you can see, we have abdicated the statistics to scipy, because it is probably less prone to error than writing our own t test.01_Registration.ipynb

We can now use this function with the data we have loaded to get a map of t values, which tells us how the dose affects outcome in different regions of the anatomy.

In [ ]:
mask = sitk.GetArrayFromImage(sitk.ReadImage("D:/Dropbox/Work/Transfer/ColabData/HNSCC_data/0002_mask.nii")).astype(np.float32)

## mask will be 0 or 1, only calculate where it is 1, so set 0 regions to nan and use nan policy from ttest_ind to ignore them
mask[mask == 0] = np.nan
print(statusArray.shape)
tMap, pMap = calculateT(doseArray, statusArray, mask=mask)

referenceAnatomy = sitk.GetArrayFromImage(sitk.ReadImage("D:/Dropbox/Work/Transfer/ColabData/HNSCC_data/downsampledCTs/0002.nii"))

fig = plt.figure(figsize=(10,10))
anatomy = plt.imshow(referenceAnatomy[::-1,64,...], cmap='Greys_r')
tMapOverlay = plt.imshow(tMap[::-1,64,...], alpha=0.5)
_ = plt.axis('off')

Now you should have an image of the patient anatomy overlaid with a t-map indicating where dose is having an effect on outcome.

This is cool, but it isn't the whole story! We need to assess the statistical significance of the values in the t-map. Because there are so many voxels in the image, we can't do this analytically. This is known as a multiple comparisons problem.

The standard way to assess significance in image based data mining is to do a permutation test. In this, we randomly shuffle the labels used to calculate the t-map, and compare the resulting permuted t-map to the original. Anywhere where the 'true' t-map is always more extreme than the permuted t-map is a statistically significant region.

Usually, we record a single 'summary statistic' from the permutations. Here we will use the simplest summary statistic possible: the most extreme value.

We can write two functions to help us do a permutation test: one to calculate the t-map for a given permutation, and another to organise the information coming from the permutations.

In [ ]:
def doPermutation(doseData, statuses, mask=None):
    """
    Permute the statuses and return the maximum t value for this permutation
    Inputs:
        - doseData: the dose data, should be structured such that the number of patients in it is along the last axis
        - statuses: the outcome labels. 1 indicates an event, 0 indicates no event. These will be permuted in this function to 
                    assess the null hypothesis of no dose interaction
    Returns:
        - (tMin, tMax): the extreme values of the whole t-value map for this permutation
    """
    pstatuses = np.random.permutation(statuses)
    permT = calculateT(doseData, pstatuses, mask=mask)
    return (np.nanmin(permT), np.nanmax(permT))

This function is pretty simple, it uses a numpy function to randomly permute the status labels, then re-calculates the t-map.

The returned values from this function are the smallest and largest values. Usually these will be negative and positive respectively, indicating a different effect of the dose on outcome.

In [ ]:
def permutationTest(doseData, statuses, nperm=1000, mask=None):
    """
    Perform a permutation test to get the global p-value and t-thresholds
    Inputs:
        - doseData: the dose data, should be structured such that the number of patients in it is along the last axis
        - statuses: the outcome labels. 1 indicates an event, 0 indicates no event.
        - nperm: The number of permutations to calculate. Defaults to 1000 which is the minimum for reasonable accuracy
    Returns:
        - globalPNeg: the global significance of the test for negative t-values
        - globalPPos: the global significance of the test for positive t-values
        - tThreshNeg: the list of minT from all the permutations, use it to set a significance threshold.
        - tThreshPos: the list of maxT from all the permutations, use it to set a significance threshold.
    """
    tthresh = []
    gtCount = 0
    ltCount = 0
    trueT = calculateT(doseData, statuses, mask=mask)
    trueMaxT = np.nanmax(trueT)
    trueMinT = np.nanmin(trueT)
    if haveTQDM:
        for perm in tqdm(range(nperm)):
            tthresh.append(doPermutation(doseData, statuses, mask=mask))
            if tthresh[-1][1] > trueMaxT:
                gtCount += 1.0
            if tthresh[-1][0] < trueMinT:
                ltCount += 1.0
    else:
        for perm in range(nperm):
            tthresh.append(doPermutation(doseData, statuses, mask=mask))
            if tthresh[-1][1] > trueMaxT:
                gtCount += 1.0
            if tthresh[-1][0] < trueMinT:
                ltCount += 1.0
    
    globalpPos = gtCount / float(nperm)
    globalpNeg = ltCount / float(nperm)
    print(gtCount, ltCount)
    tthresh = np.array(tthresh)
    return (globalpNeg, globalpPos, sorted(tthresh[:,0]), sorted(tthresh[:,1]))

This function organises the data returned from the doPermutation function, and returns us some global summary statistics, as well as an ordered array of maximum and minimum t values from the permutations, that can be used to place significance thresholds on the t-map from the true status labels.

The nperm argument to this function is important. It is the number of permutations we will perform - this impacts the minimum p-value we can see (1/nperm) but more importantly impacts the length of time the analysis takes to run. A standard number of permutations would be 1000, though this might take a while!

Let's apply the permutation test to our data!

*Warning: this cell will take a long time to run! About 6 minutes on my laptop. If you don't have tqdm for a progressbar, have faith that it is working!*

In [ ]:
pNeg, pPos, threshNeg, threshPos = permutationTest(doseArray, statusArray, nperm=10, mask=mask)

Now we have what we need to look for a dose sensitive region! To get an idea whether we have anything to look at, we can look at the global p-values. These are contained in the pNeg and pPos variables

In [ ]:
print(pNeg, pPos)


The usual threshold for saying a result is statstically sgnificant is p=<0.05. Unfortunately, in my example analysis we don't seem to have a globally significant result. Everything below here won't really work properly because there is no significant result in this case, however let's do it anyway so you can see what to do when you mine something else later and get a significant result!

---

We also have our map of T values, and the associated permutation test distribution, so we can plot the regions of significance overlaid on the t-map and CT anatomy. To do this, we use matplotlib's imshow and contourf functions as below

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

# First show the CT
ctImg = ax.imshow(referenceAnatomy[:,64,:], cmap='Greys_r')

# Now add the t-map with some transparency
tmapImg = ax.imshow(tMap[:,64,:], alpha=0.3)

plt.axis('off');

neg_p005 = np.percentile(threshNeg, 0.05)
neg_p010 = np.percentile(threshNeg, 0.10) ## Contour plot needs two levels, so we use p=0.05 & 0.10
pos_p005 = np.percentile(threshPos, 0.95)
pos_p010 = np.percentile(threshPos, 0.9)

print(neg_p005)
## Now do the contourplot at the 95% level for p=0.05
pos_contourplot = plt.contourf(tMap[:,64,:], levels=[pos_p010, pos_p005], colors='r')
neg_contourplot = plt.contourf(tMap[:,64,:], levels=[neg_p005, neg_p010], colors='g')

Now we have a complete pipeline to do image based data mining!

Now use this pipeline to try mining against some of the other outcomes in the clinical data, for example:

- Local/regional failure
- Feeding tube insertion
- Weight loss
- Skeletal muscle index reduction

Have fun!